In [44]:
import pandas as pd
import numpy as np

# import data to pandas database
synth1 = pd.read_csv('data\synthetic-1.csv', header= 0)
synth2 = pd.read_csv('data\synthetic-2.csv', header= 0)
synth3 = pd.read_csv('data\synthetic-3.csv', header= 0)
synth4 = pd.read_csv('data\synthetic-4.csv', header= 0 )

# set column titles
synth1.columns = ['Feature1', 'Feature2', 'ClassLabel']
synth2.columns = ['Feature1', 'Feature2', 'ClassLabel']
synth3.columns = ['Feature1', 'Feature2', 'ClassLabel']
synth4.columns = ['Feature1', 'Feature2', 'ClassLabel']

# append datasets together
synthMerge = synth1.append(synth2, ignore_index= True)
synthMerge = synthMerge.append(synth3, ignore_index= True)
synthMerge = synthMerge.append(synth4, ignore_index= True)

C:\Users\Luke\AppData\Local\Temp\ipykernel_25336\665881931.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  synthMerge = synth1.append(synth2, ignore_index= True)
C:\Users\Luke\AppData\Local\Temp\ipykernel_25336\665881931.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  synthMerge = synthMerge.append(synth3, ignore_index= True)
C:\Users\Luke\AppData\Local\Temp\ipykernel_25336\665881931.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  synthMerge = synthMerge.append(synth4, ignore_index= True)


In [45]:
#  Display new dataframe
synthMerge

,Feature1,Feature2,ClassLabel
0,10.81300,0.77631,1
1,8.74050,1.26280,1
2,11.18400,4.37280,1
3,12.27200,-2.39240,1
4,10.82600,2.57780,1
...,...,...,...
791,3.04470,-7.38210,0
792,6.99790,10.22000,0
793,-0.16424,-6.37990,0
794,-3.02820,-0.61758,0


In [48]:
# calculate entropy 
synthMerge['ClassLabel'].value_counts()

# count the different class labels
numOfNo = len(synthMerge[synthMerge['ClassLabel'] == 0])
numOfYes = len(synthMerge[synthMerge['ClassLabel'] == 1])
numTotal = synthMerge.shape[0]

entropy = -(numOfYes/numTotal) * np.log2(numOfYes/numTotal) - (numOfNo/numTotal) * np.log2(numOfNo/numTotal)

# Entropy = 0.9999817845368457

0.9999817845368457

In [56]:
# Threshold Offset 

f1Max = synthMerge['Feature1'].max() # 14.412
f1Min = synthMerge['Feature2'].min() # -10.616 
thresholdVal = 10

f1ThresholdOffset = (f1Max - f1Min)/thresholdVal

f1ThresholdOffset

2.5027999999999997